# Investment Comparison Tool

This notebook helps compare different investment types including:
- Stocks/ETFs
- Real Estate
- Bonds/Fixed Income

Use this to analyze returns, risks, and make informed investment decisions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Stocks/ETFs Analysis

In [ ]:
# Define stock/ETF investments
stocks = {
    'SPY': {'name': 'S&P 500 ETF', 'investment': 10000},
    'VTI': {'name': 'Total Stock Market ETF', 'investment': 10000},
    'QQQ': {'name': 'Nasdaq 100 ETF', 'investment': 10000}
}

# Fetch historical data for multiple time periods
end_date = datetime.now()
start_date_5y = end_date - timedelta(days=365*5)
start_date_10y = end_date - timedelta(days=365*10)
start_date_15y = end_date - timedelta(days=365*15)

stock_data_5y = {}
stock_data_10y = {}
stock_data_15y = {}

print("Downloading historical stock data...")
for ticker in stocks.keys():
    stock_data_5y[ticker] = yf.download(ticker, start=start_date_5y, end=end_date, progress=False)
    stock_data_10y[ticker] = yf.download(ticker, start=start_date_10y, end=end_date, progress=False)
    stock_data_15y[ticker] = yf.download(ticker, start=start_date_15y, end=end_date, progress=False)

print("Stock data downloaded successfully!")

In [ ]:
# Calculate returns and metrics
def calculate_stock_metrics(ticker, data, initial_investment):
    returns = data['Adj Close'].pct_change().dropna()
    
    initial_price = data['Adj Close'].iloc[0]
    final_price = data['Adj Close'].iloc[-1]
    total_return = (final_price - initial_price) / initial_price
    final_value = initial_investment * (1 + total_return)
    
    annual_return = returns.mean() * 252  # Annualized
    annual_volatility = returns.std() * np.sqrt(252)  # Annualized
    sharpe_ratio = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    # Calculate years of data
    years = len(data) / 252
    
    return {
        'Total Return (%)': total_return * 100,
        'Annual Return (%)': annual_return * 100,
        'Annual Volatility (%)': annual_volatility * 100,
        'Sharpe Ratio': sharpe_ratio,
        'Initial Investment': initial_investment,
        'Final Value': final_value,
        'Gain/Loss': final_value - initial_investment,
        'Years': years
    }

# Calculate metrics for 5-year period
stock_metrics = {}
for ticker, info in stocks.items():
    stock_metrics[info['name']] = calculate_stock_metrics(ticker, stock_data_5y[ticker], info['investment'])

stock_df = pd.DataFrame(stock_metrics).T
print("\nStock/ETF Performance Metrics (5 Years):")
stock_df

In [ ]:
# Plot stock performance
plt.figure(figsize=(14, 6))

for ticker in stocks.keys():
    normalized = stock_data[ticker]['Adj Close'] / stock_data[ticker]['Adj Close'].iloc[0]
    plt.plot(normalized, label=stocks[ticker]['name'])

plt.title('Normalized Stock/ETF Performance', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Normalized Price (Base = 1.0)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Year-by-Year Performance Analysis
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# 1. Annual Returns by Year
ax1 = axes[0]
annual_returns_data = []

for ticker, info in stocks.items():
    prices = stock_data_5y[ticker]['Adj Close']
    # Group by year and calculate annual returns
    yearly_data = prices.resample('Y').last()
    yearly_returns = yearly_data.pct_change().dropna() * 100
    
    for date, ret in yearly_returns.items():
        annual_returns_data.append({
            'Investment': info['name'],
            'Year': date.year,
            'Return (%)': ret
        })

annual_df = pd.DataFrame(annual_returns_data)
pivot_annual = annual_df.pivot(index='Year', columns='Investment', values='Return (%)')

pivot_annual.plot(kind='bar', ax=ax1, width=0.8, colormap='Set2')
ax1.set_title('Annual Returns by Year', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Annual Return (%)', fontsize=12)
ax1.legend(title='Investment', bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3, axis='y')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax1.tick_params(axis='x', rotation=45)

# 2. Monthly Returns Heatmap (for first stock as example)
ax2 = axes[1]
ticker = list(stocks.keys())[0]
info = stocks[ticker]
prices = stock_data_5y[ticker]['Adj Close']

# Calculate monthly returns
monthly_returns = prices.resample('M').last().pct_change() * 100
monthly_returns.index = pd.to_datetime(monthly_returns.index)

# Create pivot table for heatmap
heatmap_data = pd.pivot_table(
    pd.DataFrame({'Return': monthly_returns.values, 
                  'Year': monthly_returns.index.year,
                  'Month': monthly_returns.index.month}),
    values='Return',
    index='Month',
    columns='Year'
)

# Plot heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
            ax=ax2, cbar_kws={'label': 'Return (%)'}, linewidths=0.5)
ax2.set_title(f'Monthly Returns Heatmap - {info["name"]}', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Month', fontsize=12)
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax2.set_yticklabels(month_labels, rotation=0)

plt.tight_layout()
plt.show()

# Print annual performance summary
print("\n" + "="*80)
print("ANNUAL PERFORMANCE SUMMARY")
print("="*80)
print("\nAverage Annual Returns:")
for col in pivot_annual.columns:
    avg_return = pivot_annual[col].mean()
    best_year = pivot_annual[col].idxmax()
    worst_year = pivot_annual[col].idxmin()
    print(f"\n{col}:")
    print(f"  Average: {avg_return:.2f}%")
    print(f"  Best Year: {best_year} ({pivot_annual.loc[best_year, col]:.2f}%)")
    print(f"  Worst Year: {worst_year} ({pivot_annual.loc[worst_year, col]:.2f}%)")
    print(f"  Positive Years: {(pivot_annual[col] > 0).sum()} / {len(pivot_annual)}")
print("="*80)

In [ ]:
# Volatility and Correlation Analysis
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# 1. Rolling Volatility (30-day)
ax1 = axes[0]
for ticker, info in stocks.items():
    returns = stock_data_5y[ticker]['Adj Close'].pct_change()
    rolling_vol = returns.rolling(window=30).std() * np.sqrt(252) * 100
    ax1.plot(rolling_vol.index, rolling_vol, label=info['name'], linewidth=2)

ax1.set_title('Rolling 30-Day Volatility', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Annualized Volatility (%)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Correlation Heatmap
ax2 = axes[1]
# Create returns dataframe
returns_df = pd.DataFrame()
for ticker, info in stocks.items():
    returns_df[info['name']] = stock_data_5y[ticker]['Adj Close'].pct_change()

correlation_matrix = returns_df.corr()
im = ax2.imshow(correlation_matrix, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)

# Add colorbar
cbar = plt.colorbar(im, ax=ax2)
cbar.set_label('Correlation', fontsize=10)

# Set ticks and labels
ax2.set_xticks(np.arange(len(correlation_matrix.columns)))
ax2.set_yticks(np.arange(len(correlation_matrix.index)))
ax2.set_xticklabels(correlation_matrix.columns, rotation=45, ha='right')
ax2.set_yticklabels(correlation_matrix.index)

# Add correlation values as text
for i in range(len(correlation_matrix)):
    for j in range(len(correlation_matrix.columns)):
        text = ax2.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=11, fontweight='bold')

ax2.set_title('Return Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("CORRELATION INSIGHTS:")
print("="*80)
print("Correlation measures how investments move together:")
print("  • 1.0  = Perfect positive correlation (move exactly together)")
print("  • 0.0  = No correlation (independent movements)")
print("  • -1.0 = Perfect negative correlation (move in opposite directions)")
print("\nFor diversification, look for correlations < 0.7")
print("="*80)

In [ ]:
# Return Distribution Analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 10))

for idx, (ticker, info) in enumerate(stocks.items()):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col] if len(stocks) > 3 else axes[col]
    
    # Calculate daily returns
    returns = stock_data_5y[ticker]['Adj Close'].pct_change().dropna() * 100
    
    # Create histogram
    ax.hist(returns, bins=50, alpha=0.7, color='#3498db', edgecolor='black')
    
    # Add normal distribution overlay
    mu = returns.mean()
    sigma = returns.std()
    x = np.linspace(returns.min(), returns.max(), 100)
    ax.plot(x, len(returns) * (returns.max() - returns.min()) / 50 * 
            (1/(sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2),
            'r-', linewidth=2, label='Normal Distribution')
    
    # Add vertical line for mean
    ax.axvline(mu, color='green', linestyle='--', linewidth=2, label=f'Mean: {mu:.2f}%')
    ax.axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    
    ax.set_title(f'{info["name"]}\nDaily Returns Distribution', fontsize=12, fontweight='bold')
    ax.set_xlabel('Daily Return (%)', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

# Remove empty subplots if less than 6 stocks
for idx in range(len(stocks), 6):
    row = idx // 3
    col = idx % 3
    fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()

# Print distribution statistics
print("\n" + "="*80)
print("RETURN DISTRIBUTION STATISTICS (5 Years)")
print("="*80)
for ticker, info in stocks.items():
    returns = stock_data_5y[ticker]['Adj Close'].pct_change().dropna() * 100
    print(f"\n{info['name']}:")
    print(f"  Mean Daily Return: {returns.mean():.3f}%")
    print(f"  Median Daily Return: {returns.median():.3f}%")
    print(f"  Std Deviation: {returns.std():.3f}%")
    print(f"  Skewness: {returns.skew():.3f}")
    print(f"  Best Day: {returns.max():.2f}%")
    print(f"  Worst Day: {returns.min():.2f}%")

In [ ]:
# Growth of $10,000 investment over time
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1. Portfolio Growth Over Time (5 years)
ax1 = axes[0, 0]
for ticker, info in stocks.items():
    initial_investment = info['investment']
    prices = stock_data_5y[ticker]['Adj Close']
    portfolio_value = (prices / prices.iloc[0]) * initial_investment
    ax1.plot(portfolio_value.index, portfolio_value, label=info['name'], linewidth=2)

ax1.set_title('Portfolio Value Growth - 5 Years', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axhline(y=10000, color='red', linestyle='--', alpha=0.5, label='Initial Investment')

# 2. Cumulative Returns (%)
ax2 = axes[0, 1]
for ticker, info in stocks.items():
    prices = stock_data_5y[ticker]['Adj Close']
    cumulative_returns = (prices / prices.iloc[0] - 1) * 100
    ax2.plot(cumulative_returns.index, cumulative_returns, label=info['name'], linewidth=2)

ax2.set_title('Cumulative Returns - 5 Years', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Cumulative Return (%)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# 3. Rolling 6-Month Returns
ax3 = axes[1, 0]
for ticker, info in stocks.items():
    returns = stock_data_5y[ticker]['Adj Close'].pct_change()
    rolling_returns = returns.rolling(window=126).apply(lambda x: (1 + x).prod() - 1) * 100
    ax3.plot(rolling_returns.index, rolling_returns, label=info['name'], linewidth=2)

ax3.set_title('Rolling 6-Month Returns', fontsize=14, fontweight='bold')
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('6-Month Return (%)', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# 4. Drawdown Analysis (Peak to Trough)
ax4 = axes[1, 1]
for ticker, info in stocks.items():
    prices = stock_data_5y[ticker]['Adj Close']
    cumulative = prices / prices.iloc[0]
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max * 100
    ax4.plot(drawdown.index, drawdown, label=info['name'], linewidth=2)

ax4.set_title('Drawdown from Peak - 5 Years', fontsize=14, fontweight='bold')
ax4.set_xlabel('Date', fontsize=12)
ax4.set_ylabel('Drawdown (%)', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.fill_between(drawdown.index, drawdown.min(), 0, alpha=0.1, color='red')

plt.tight_layout()
plt.show()

### Detailed Growth Charts

In [ ]:
# Visualize long-term returns
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Annual returns by period
pivot_returns = long_term_df.pivot(index='Investment', columns='Period', values='Annual Return (%)')
pivot_returns.plot(kind='bar', ax=axes[0], color=['#3498db', '#2980b9'], width=0.8)
axes[0].set_title('Average Annual Returns by Time Period', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Investment', fontsize=12)
axes[0].set_ylabel('Annual Return (%)', fontsize=12)
axes[0].legend(title='Period')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].tick_params(axis='x', rotation=45)

# Sharpe ratios by period
pivot_sharpe = long_term_df.pivot(index='Investment', columns='Period', values='Sharpe Ratio')
pivot_sharpe.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#27ae60'], width=0.8)
axes[1].set_title('Sharpe Ratios by Time Period', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Investment', fontsize=12)
axes[1].set_ylabel('Sharpe Ratio', fontsize=12)
axes[1].legend(title='Period')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=45)
axes[1].axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Good (>1.0)')

plt.tight_layout()
plt.show()

In [ ]:
# Define real estate investments
real_estate_investments = [
    {
        'name': 'Rental Property A',
        'purchase_price': 250000,
        'down_payment': 50000,
        'monthly_rent': 2000,
        'monthly_expenses': 600,  # mortgage, taxes, insurance, maintenance
        'appreciation_rate': 0.03,  # 3% annual
        'years': 5
    },
    {
        'name': 'REIT Investment',
        'purchase_price': 50000,
        'down_payment': 50000,
        'monthly_rent': 200,  # dividend income
        'monthly_expenses': 0,
        'appreciation_rate': 0.05,  # 5% annual
        'years': 5
    }
]

def calculate_real_estate_metrics(investment):
    monthly_cash_flow = investment['monthly_rent'] - investment['monthly_expenses']
    annual_cash_flow = monthly_cash_flow * 12
    total_cash_flow = annual_cash_flow * investment['years']
    
    future_value = investment['purchase_price'] * (1 + investment['appreciation_rate']) ** investment['years']
    equity_gain = future_value - investment['purchase_price']
    
    total_return = total_cash_flow + equity_gain
    roi = (total_return / investment['down_payment']) * 100
    annual_roi = roi / investment['years']
    
    # Estimate volatility for real estate (typically 10-15% for properties, 15-20% for REITs)
    if 'REIT' in investment['name']:
        volatility = 18.0  # REITs have higher volatility
    else:
        volatility = 12.0  # Direct property ownership
    
    # Calculate Sharpe ratio (assuming risk-free rate of 3%)
    risk_free_rate = 3.0
    sharpe_ratio = (annual_roi - risk_free_rate) / volatility
    
    return {
        'Initial Investment': investment['down_payment'],
        'Monthly Cash Flow': monthly_cash_flow,
        'Total Cash Flow': total_cash_flow,
        'Property Appreciation': equity_gain,
        'Total Return': total_return,
        'ROI (%)': roi,
        'Annual ROI (%)': annual_roi,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Final Value': investment['down_payment'] + total_return
    }

real_estate_metrics = {inv['name']: calculate_real_estate_metrics(inv) for inv in real_estate_investments}
real_estate_df = pd.DataFrame(real_estate_metrics).T

print("\nReal Estate Investment Metrics:")
real_estate_df

In [ ]:
# Comprehensive All-Investment Comparison Dashboard
fig = plt.figure(figsize=(20, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

colors_map = {'Stock/ETF': '#3498db', 'Real Estate': '#2ecc71', 'Bond/Fixed Income': '#e74c3c'}

# 1. Investment Value Comparison (Bubble Chart)
ax1 = fig.add_subplot(gs[0, 0])
for inv_type in comparison_df['Type'].unique():
    subset = comparison_df[comparison_df['Type'] == inv_type]
    ax1.scatter(subset['Initial Investment'], subset['Final Value'], 
               s=subset['Sharpe Ratio']*200, alpha=0.6, 
               color=colors_map[inv_type], label=inv_type, edgecolors='black', linewidth=2)

# Add diagonal line (break-even)
max_val = max(comparison_df['Initial Investment'].max(), comparison_df['Final Value'].max())
ax1.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, linewidth=2, label='Break-even')

ax1.set_xlabel('Initial Investment ($)', fontsize=11)
ax1.set_ylabel('Final Value ($)', fontsize=11)
ax1.set_title('Investment Growth\n(Bubble size = Sharpe Ratio)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# 2. Return Distribution by Type (Violin Plot)
ax2 = fig.add_subplot(gs[0, 1])
types = comparison_df['Type'].unique()
positions = np.arange(len(types))

for i, inv_type in enumerate(types):
    subset = comparison_df[comparison_df['Type'] == inv_type]
    parts = ax2.violinplot([subset['Annual Return (%)'].values], positions=[i], 
                          showmeans=True, showextrema=True, widths=0.7)
    for pc in parts['bodies']:
        pc.set_facecolor(colors_map[inv_type])
        pc.set_alpha(0.7)

ax2.set_xticks(positions)
ax2.set_xticklabels(types, rotation=15, ha='right')
ax2.set_ylabel('Annual Return (%)', fontsize=11)
ax2.set_title('Return Distribution by Type', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# 3. Gain/Loss Waterfall Chart
ax3 = fig.add_subplot(gs[0, 2])
sorted_comp = comparison_df.sort_values('Gain/Loss', ascending=False)
colors_list = [colors_map[t] for t in sorted_comp['Type']]

bars = ax3.bar(range(len(sorted_comp)), sorted_comp['Gain/Loss'], color=colors_list, edgecolor='black')
ax3.set_xticks(range(len(sorted_comp)))
ax3.set_xticklabels(sorted_comp['Investment'], rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('Gain/Loss ($)', fontsize=11)
ax3.set_title('Absolute Gains/Losses', fontsize=12, fontweight='bold')
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.grid(True, alpha=0.3, axis='y')

# 4. Efficiency Frontier (Risk vs Return with trend line)
ax4 = fig.add_subplot(gs[1, 0])
for inv_type in comparison_df['Type'].unique():
    subset = comparison_df[comparison_df['Type'] == inv_type]
    ax4.scatter(subset['Risk (Volatility %)'], subset['Annual Return (%)'],
               s=200, alpha=0.7, color=colors_map[inv_type], 
               label=inv_type, edgecolors='black', linewidth=2)
    
    # Add labels for each point
    for idx, row in subset.iterrows():
        ax4.annotate(row['Investment'], 
                    (row['Risk (Volatility %)'], row['Annual Return (%)']),
                    fontsize=8, ha='right', va='bottom', alpha=0.8)

# Add efficient frontier line
sorted_by_risk = comparison_df.sort_values('Risk (Volatility %)')
ax4.plot(sorted_by_risk['Risk (Volatility %)'], 
        sorted_by_risk['Annual Return (%)'], 
        'k--', alpha=0.3, linewidth=1, label='Trend')

ax4.set_xlabel('Risk (Volatility %)', fontsize=11)
ax4.set_ylabel('Annual Return (%)', fontsize=11)
ax4.set_title('Efficient Frontier Analysis', fontsize=12, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

# 5. Portfolio Allocation Comparison
ax5 = fig.add_subplot(gs[1, 1])
type_totals = comparison_df.groupby('Type')['Initial Investment'].sum()
explode = [0.05] * len(type_totals)
colors_pie = [colors_map[t] for t in type_totals.index]

wedges, texts, autotexts = ax5.pie(type_totals, labels=type_totals.index, autopct='%1.1f%%',
                                    colors=colors_pie, explode=explode, startangle=90,
                                    textprops={'fontsize': 10, 'weight': 'bold'})
ax5.set_title('Current Portfolio Allocation\n(by Initial Investment)', fontsize=12, fontweight='bold')

# 6. Performance Metrics Radar Chart
ax6 = fig.add_subplot(gs[1, 2], projection='polar')

# Prepare data for radar chart
categories = ['Return', 'Sharpe', 'Low Risk']
type_avg = comparison_df.groupby('Type').agg({
    'Annual Return (%)': 'mean',
    'Sharpe Ratio': 'mean',
    'Risk (Volatility %)': lambda x: 100 - x.mean()  # Invert so higher is better
})

# Normalize to 0-100 scale
for col in type_avg.columns:
    type_avg[col] = (type_avg[col] - type_avg[col].min()) / (type_avg[col].max() - type_avg[col].min()) * 100

angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

for inv_type in type_avg.index:
    values = type_avg.loc[inv_type].tolist()
    values += values[:1]
    ax6.plot(angles, values, 'o-', linewidth=2, label=inv_type, color=colors_map[inv_type])
    ax6.fill(angles, values, alpha=0.15, color=colors_map[inv_type])

ax6.set_xticks(angles[:-1])
ax6.set_xticklabels(categories, fontsize=10)
ax6.set_ylim(0, 100)
ax6.set_title('Investment Type Comparison\n(Normalized)', fontsize=12, fontweight='bold', pad=20)
ax6.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
ax6.grid(True)

plt.suptitle('COMPREHENSIVE INVESTMENT COMPARISON DASHBOARD', 
            fontsize=16, fontweight='bold', y=0.995)
plt.show()

### Long-Term Performance Analysis (10 & 15 Years)

# Define bond investments
bonds = [
    {
        'name': '10-Year Treasury',
        'principal': 50000,
        'annual_rate': 0.04,  # 4% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'Corporate Bond',
        'principal': 50000,
        'annual_rate': 0.055,  # 5.5% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'CD (Certificate of Deposit)',
        'principal': 50000,
        'annual_rate': 0.045,  # 4.5% APY
        'years': 5,
        'compounding': 'monthly'
    }
]

def calculate_bond_metrics(bond):
    # Determine compounding frequency
    if bond['compounding'] == 'annual':
        n = 1
    elif bond['compounding'] == 'semi-annual':
        n = 2
    elif bond['compounding'] == 'quarterly':
        n = 4
    elif bond['compounding'] == 'monthly':
        n = 12
    else:
        n = 1
    
    # Calculate future value with compound interest
    future_value = bond['principal'] * (1 + bond['annual_rate'] / n) ** (n * bond['years'])
    total_interest = future_value - bond['principal']
    total_return_pct = (total_interest / bond['principal']) * 100
    annual_return = total_return_pct / bond['years']
    
    # Estimate volatility based on bond type
    if 'Treasury' in bond['name']:
        volatility = 2.5  # Very low for government bonds
    elif 'Corporate' in bond['name']:
        volatility = 5.0  # Moderate for corporate bonds
    else:
        volatility = 1.0  # Minimal for CDs (FDIC insured)
    
    # Calculate Sharpe ratio (assuming risk-free rate of 3%)
    risk_free_rate = 3.0
    sharpe_ratio = (annual_return - risk_free_rate) / volatility if volatility > 0 else 0
    
    return {
        'Initial Investment': bond['principal'],
        'Annual Rate (%)': bond['annual_rate'] * 100,
        'Years': bond['years'],
        'Total Interest': total_interest,
        'Final Value': future_value,
        'Total Return (%)': total_return_pct,
        'Annual Return (%)': annual_return,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe_ratio
    }

bond_metrics = {bond['name']: calculate_bond_metrics(bond) for bond in bonds}
bond_df = pd.DataFrame(bond_metrics).T

print("\nBonds/Fixed Income Metrics:")
bond_df

In [ ]:
# Define real estate investments
real_estate_investments = [
    {
        'name': 'Rental Property A',
        'purchase_price': 250000,
        'down_payment': 50000,
        'monthly_rent': 2000,
        'monthly_expenses': 600,  # mortgage, taxes, insurance, maintenance
        'appreciation_rate': 0.03,  # 3% annual
        'years': 5
    },
    {
        'name': 'REIT Investment',
        'purchase_price': 50000,
        'down_payment': 50000,
        'monthly_rent': 200,  # dividend income
        'monthly_expenses': 0,
        'appreciation_rate': 0.05,  # 5% annual
        'years': 5
    }
]

def calculate_real_estate_metrics(investment):
    monthly_cash_flow = investment['monthly_rent'] - investment['monthly_expenses']
    annual_cash_flow = monthly_cash_flow * 12
    total_cash_flow = annual_cash_flow * investment['years']
    
    future_value = investment['purchase_price'] * (1 + investment['appreciation_rate']) ** investment['years']
    equity_gain = future_value - investment['purchase_price']
    
    total_return = total_cash_flow + equity_gain
    roi = (total_return / investment['down_payment']) * 100
    annual_roi = roi / investment['years']
    
    return {
        'Initial Investment': investment['down_payment'],
        'Monthly Cash Flow': monthly_cash_flow,
        'Total Cash Flow': total_cash_flow,
        'Property Appreciation': equity_gain,
        'Total Return': total_return,
        'ROI (%)': roi,
        'Annual ROI (%)': annual_roi,
        'Final Value': investment['down_payment'] + total_return
    }

real_estate_metrics = {inv['name']: calculate_real_estate_metrics(inv) for inv in real_estate_investments}
real_estate_df = pd.DataFrame(real_estate_metrics).T

print("\nReal Estate Investment Metrics:")
real_estate_df

# Combine all investments for comparison
comparison_data = []

# Add stocks
for name, metrics in stock_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Stock/ETF',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Gain/Loss'],
        'Return (%)': metrics['Total Return (%)'],
        'Annual Return (%)': metrics['Annual Return (%)'],
        'Risk (Volatility %)': metrics['Annual Volatility (%)'],
        'Sharpe Ratio': metrics['Sharpe Ratio']
    })

# Add real estate
for name, metrics in real_estate_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Real Estate',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Return'],
        'Return (%)': metrics['ROI (%)'],
        'Annual Return (%)': metrics['Annual ROI (%)'],
        'Risk (Volatility %)': metrics['Volatility (%)'],
        'Sharpe Ratio': metrics['Sharpe Ratio']
    })

# Add bonds
for name, metrics in bond_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Bond/Fixed Income',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Interest'],
        'Return (%)': metrics['Total Return (%)'],
        'Annual Return (%)': metrics['Annual Return (%)'],
        'Risk (Volatility %)': metrics['Volatility (%)'],
        'Sharpe Ratio': metrics['Sharpe Ratio']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Return (%)', ascending=False)

print("\nComprehensive Investment Comparison:")
print("=" * 80)
comparison_df

In [ ]:
# Define bond investments
bonds = [
    {
        'name': '10-Year Treasury',
        'principal': 50000,
        'annual_rate': 0.04,  # 4% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'Corporate Bond',
        'principal': 50000,
        'annual_rate': 0.055,  # 5.5% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'CD (Certificate of Deposit)',
        'principal': 50000,
        'annual_rate': 0.045,  # 4.5% APY
        'years': 5,
        'compounding': 'monthly'
    }
]

def calculate_bond_metrics(bond):
    # Determine compounding frequency
    if bond['compounding'] == 'annual':
        n = 1
    elif bond['compounding'] == 'semi-annual':
        n = 2
    elif bond['compounding'] == 'quarterly':
        n = 4
    elif bond['compounding'] == 'monthly':
        n = 12
    else:
        n = 1
    
    # Calculate future value with compound interest
    future_value = bond['principal'] * (1 + bond['annual_rate'] / n) ** (n * bond['years'])
    total_interest = future_value - bond['principal']
    total_return_pct = (total_interest / bond['principal']) * 100
    annual_return = total_return_pct / bond['years']
    
    return {
        'Initial Investment': bond['principal'],
        'Annual Rate (%)': bond['annual_rate'] * 100,
        'Years': bond['years'],
        'Total Interest': total_interest,
        'Final Value': future_value,
        'Total Return (%)': total_return_pct,
        'Annual Return (%)': annual_return
    }

bond_metrics = {bond['name']: calculate_bond_metrics(bond) for bond in bonds}
bond_df = pd.DataFrame(bond_metrics).T

print("\nBonds/Fixed Income Metrics:")
bond_df

In [ ]:
# Enhanced summary statistics
print("\n" + "="*80)
print("COMPREHENSIVE INVESTMENT SUMMARY")
print("="*80)

print(f"\nTotal Capital Invested: ${comparison_df['Initial Investment'].sum():,.2f}")
print(f"Total Portfolio Value: ${comparison_df['Final Value'].sum():,.2f}")
print(f"Total Gain/Loss: ${comparison_df['Gain/Loss'].sum():,.2f}")
print(f"Overall Return: {(comparison_df['Gain/Loss'].sum() / comparison_df['Initial Investment'].sum() * 100):.2f}%")

print("\n" + "-"*80)
print("TOP PERFORMERS BY METRIC:")
print("-"*80)

# Best total return
best_return = comparison_df.loc[comparison_df['Return (%)'].idxmax()]
print(f"\nHighest Total Return: {best_return['Investment']}")
print(f"  Return: {best_return['Return (%)']:.2f}%")
print(f"  Type: {best_return['Type']}")

# Best annual return
best_annual = comparison_df.loc[comparison_df['Annual Return (%)'].idxmax()]
print(f"\nHighest Annual Return: {best_annual['Investment']}")
print(f"  Annual Return: {best_annual['Annual Return (%)']:.2f}%")
print(f"  Type: {best_annual['Type']}")

# Best Sharpe ratio
best_sharpe = comparison_df.loc[comparison_df['Sharpe Ratio'].idxmax()]
print(f"\nBest Risk-Adjusted Returns (Highest Sharpe): {best_sharpe['Investment']}")
print(f"  Sharpe Ratio: {best_sharpe['Sharpe Ratio']:.3f}")
print(f"  Annual Return: {best_sharpe['Annual Return (%)']:.2f}%")
print(f"  Volatility: {best_sharpe['Risk (Volatility %)']:.2f}%")
print(f"  Type: {best_sharpe['Type']}")

# Lowest risk
lowest_risk = comparison_df.loc[comparison_df['Risk (Volatility %)'].idxmin()]
print(f"\nLowest Risk Investment: {lowest_risk['Investment']}")
print(f"  Volatility: {lowest_risk['Risk (Volatility %)']:.2f}%")
print(f"  Annual Return: {lowest_risk['Annual Return (%)']:.2f}%")
print(f"  Type: {lowest_risk['Type']}")

print("\n" + "-"*80)
print("AVERAGE METRICS BY INVESTMENT TYPE:")
print("-"*80)

type_stats = comparison_df.groupby('Type').agg({
    'Annual Return (%)': ['mean', 'std'],
    'Risk (Volatility %)': 'mean',
    'Sharpe Ratio': 'mean'
}).round(2)

print(type_stats.to_string())

print("\n" + "="*80)
print("INVESTMENT RECOMMENDATIONS:")
print("="*80)
print("\n1. AGGRESSIVE GROWTH: Consider stocks/ETFs with highest Sharpe ratios")
print("   - Higher returns, higher volatility")
print("   - Best for long-term investors (10+ years)")
print("\n2. BALANCED PORTFOLIO: Mix of stocks, real estate, and bonds")
print("   - Diversification reduces overall risk")
print("   - Targets steady growth with moderate volatility")
print("\n3. CONSERVATIVE INCOME: Focus on bonds and stable real estate")
print("   - Lower returns but much lower risk")
print("   - Best for capital preservation and steady income")
print("="*80)

## 4. Comprehensive Comparison

In [ ]:
# Combine all investments for comparison
comparison_data = []

# Add stocks
for name, metrics in stock_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Stock/ETF',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Gain/Loss'],
        'Return (%)': metrics['Total Return (%)'],
        'Risk (Volatility %)': metrics['Annual Volatility (%)']
    })

# Add real estate
for name, metrics in real_estate_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Real Estate',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Return'],
        'Return (%)': metrics['ROI (%)'],
        'Risk (Volatility %)': 10  # Estimated for real estate
    })

# Add bonds
for name, metrics in bond_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Bond/Fixed Income',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Interest'],
        'Return (%)': metrics['Total Return (%)'],
        'Risk (Volatility %)': 2  # Low risk for bonds
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Return (%)', ascending=False)

print("\nComprehensive Investment Comparison:")
comparison_df

In [ ]:
# Visualization: Returns Comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of returns
colors = {'Stock/ETF': '#3498db', 'Real Estate': '#2ecc71', 'Bond/Fixed Income': '#e74c3c'}
comparison_df['Color'] = comparison_df['Type'].map(colors)

axes[0].barh(comparison_df['Investment'], comparison_df['Return (%)'], color=comparison_df['Color'])
axes[0].set_xlabel('Return (%)', fontsize=12)
axes[0].set_title('Investment Returns Comparison', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Risk vs Return scatter plot
for inv_type, color in colors.items():
    subset = comparison_df[comparison_df['Type'] == inv_type]
    axes[1].scatter(subset['Risk (Volatility %)'], subset['Return (%)'], 
                   c=color, label=inv_type, s=200, alpha=0.7, edgecolors='black')

axes[1].set_xlabel('Risk (Volatility %)', fontsize=12)
axes[1].set_ylabel('Return (%)', fontsize=12)
axes[1].set_title('Risk vs Return Profile', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
print("\n=== Investment Summary ===")
print(f"\nTotal Invested: ${comparison_df['Initial Investment'].sum():,.2f}")
print(f"Total Final Value: ${comparison_df['Final Value'].sum():,.2f}")
print(f"Total Gain/Loss: ${comparison_df['Gain/Loss'].sum():,.2f}")
print(f"\nBest Performing Investment: {comparison_df.iloc[0]['Investment']}")
print(f"Return: {comparison_df.iloc[0]['Return (%)']:.2f}%")

print("\n=== Risk-Adjusted Returns (Sharpe-like Ratio) ===")
comparison_df['Risk-Adjusted Return'] = comparison_df['Return (%)'] / comparison_df['Risk (Volatility %)']
best_risk_adjusted = comparison_df.sort_values('Risk-Adjusted Return', ascending=False).iloc[0]
print(f"Best Risk-Adjusted Investment: {best_risk_adjusted['Investment']}")
print(f"Risk-Adjusted Score: {best_risk_adjusted['Risk-Adjusted Return']:.2f}")

## 5. Custom Investment Analysis

Use the cells below to add your own investments and analyze them.

In [ ]:
# Add your custom investment here
# Example:
# custom_investment = {
#     'name': 'My Investment',
#     'initial': 10000,
#     'final': 12000,
#     'years': 3
# }
